In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, \
                            recall_score, f1_score, log_loss

from tqdm.notebook import tqdm

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import joblib


RAND=42
N_FOLDS = 5

In [2]:
def get_metrics(y_test, y_pred, y_score, name = "Default"):
    """Метрики для задачи классификации"""
    df_metrics = pd.DataFrame()

    df_metrics['model'] = [name]
    df_metrics['Accuracy'] = accuracy_score(y_test, y_pred)
    df_metrics['ROC_AUC'] = roc_auc_score(y_test, y_score[:, 1])
    df_metrics['Precision'] = precision_score(y_test, y_pred)
    df_metrics['Recall'] = recall_score(y_test, y_pred)
    df_metrics['f1'] = f1_score(y_test, y_pred)
    df_metrics['Logloss'] = log_loss(y_test, y_score)

    return df_metrics

# Feature engineering

In [3]:
filepath = "../data/transformed/train_data/train_data_0.pq.0"
X_0 = pd.read_parquet(filepath).set_index("id")
X_0[:5]

,enc_paym_23_2,pre_maxover2limit_2,pre_loans6090_1,enc_loans_credit_status_5,pre_fterm_11,pre_loans_outstanding_2,enc_loans_credit_type_6,pre_till_pclose_1,is_zero_util_1,pre_loans90_13,...,enc_paym_1_3,enc_loans_account_holder_type_2,rn,pre_maxover2limit_11,pre_till_pclose_5,pre_since_opened_0,pre_since_opened_11,pre_since_opened_14,enc_paym_17_2,pre_pterm_16
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,1,0,1,6,0,...,0,0,10,0,0,0,0,0,0,0
1,0,0,0,0,1,2,0,1,10,0,...,3,0,14,0,0,0,0,3,0,0
2,0,0,0,0,0,1,0,2,1,0,...,1,0,3,1,0,0,0,0,0,0
3,0,0,0,0,1,1,0,5,8,0,...,1,0,15,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,1,1,0,...,1,0,1,0,0,0,0,0,0,0


In [4]:
filepath = "../data/transformed/train_data/train_data_0.pq.1"
X_1 = pd.read_parquet(filepath).set_index("id")
X_1[:5]

,enc_paym_23_2,pre_maxover2limit_2,pre_loans6090_1,enc_loans_credit_status_5,pre_fterm_11,pre_loans_outstanding_2,enc_loans_credit_type_6,pre_till_pclose_1,is_zero_util_1,pre_loans90_13,...,enc_paym_1_3,enc_loans_account_holder_type_2,rn,pre_maxover2limit_11,pre_till_pclose_5,pre_since_opened_0,pre_since_opened_11,pre_since_opened_14,enc_paym_17_2,pre_pterm_16
id,,,,,,,,,,,,,,,,,,,,,
125000,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
125001,0,0,0,0,0,0,0,0,3,0,...,2,0,3,0,0,0,0,0,0,0
125002,0,0,0,0,0,0,0,1,6,0,...,0,0,9,0,0,0,1,0,0,1
125003,0,0,0,0,2,0,0,1,3,0,...,0,0,6,0,0,0,0,0,0,0
125004,0,0,0,0,1,0,0,1,5,0,...,1,0,5,0,1,0,0,0,0,0


In [5]:
X = pd.concat([X_0, X_1])

In [6]:
X.shape

(250000, 420)

In [7]:
target_path = "../data/raw/train_target.csv"
Y = pd.read_csv(target_path, nrows=250000, index_col="id")
Y[:5]

,flag
id,
0,0
1,0
2,0
3,0
4,0


In [8]:
Y.shape

(250000, 1)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y, 
                                                    random_state=RAND)

# Baseline models

In [ ]:
# Создаем и обучаем бейслайн LightGBM
model = LGBMClassifier(n_estimators=1000, class_weight='balanced', random_state=RAND)
model.fit(x_train, y_train)

In [12]:
y_pred_tr = model.predict(x_train)
y_score_tr = model.predict_proba(x_train)

y_pred = model.predict(x_test)
y_score = model.predict_proba(x_test)

In [13]:
res = get_metrics(y_train, y_pred_tr, y_score_tr, name="lgbm_train")
res = res.append(get_metrics(y_test, y_pred, y_score, name="lgbm_test"))
res

/tmp/ipykernel_7215/429328251.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_test, y_pred, y_score, name="lgbm_test"))


,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,lgbm_train,0.85537,0.956167,0.166378,0.916438,0.281627,0.337983
0,lgbm_test,0.82830,0.708052,0.079067,0.427279,0.133441,0.380872


In [ ]:
ratio = y_train[y_train==0].shape[0] / y_train[y_train==1].shape[0]

# Создаем и обучаем бейслайн Catboost
model = CatBoostClassifier(n_estimators=1000, scale_pos_weight=ratio, random_state=RAND, verbose=0)
model.fit(x_train, y_train)

In [22]:
y_pred_tr = model.predict(x_train)
y_score_tr = model.predict_proba(x_train)

y_pred = model.predict(x_test)
y_score = model.predict_proba(x_test)

In [23]:
res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, name="catboost_train"))
res = res.append(get_metrics(y_test, y_pred, y_score, name="catboost_test"))
res

/tmp/ipykernel_7215/787895992.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, name="catboost_train"))
/tmp/ipykernel_7215/787895992.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_test, y_pred, y_score, name="catboost_test"))


,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,lgbm_train,0.855370,0.956167,0.166378,0.916438,0.281627,0.337983
0,lgbm_test,0.828300,0.708052,0.079067,0.427279,0.133441,0.380872
0,catboost_train,0.970535,0.825089,1.000000,0.047519,0.090727,0.110776
0,catboost_test,0.968860,0.756799,0.000000,0.000000,0.000000,0.124916


In [28]:
model = RandomForestClassifier(n_estimators=1000, random_state=RAND, verbose=0)
model.fit(x_train, y_train)

/tmp/ipykernel_7215/1478570180.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


RandomForestClassifier(random_state=42)

In [29]:
y_pred_tr = model.predict(x_train)
y_score_tr = model.predict_proba(x_train)

y_pred = model.predict(x_test)
y_score = model.predict_proba(x_test)

In [30]:
res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, "LG_train"))
res = res.append(get_metrics(y_test, y_pred, y_score, "LG_test"))
res

/tmp/ipykernel_7215/3198375833.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, "LG_train"))
/tmp/ipykernel_7215/3198375833.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_test, y_pred, y_score, "LG_test"))


,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,lgbm_train,0.855370,0.956167,0.166378,0.916438,0.281627,0.337983
0,lgbm_test,0.828300,0.708052,0.079067,0.427279,0.133441,0.380872
0,catboost_train,0.970535,0.825089,1.000000,0.047519,0.090727,0.110776
0,catboost_test,0.968860,0.756799,0.000000,0.000000,0.000000,0.124916
0,LG_train,0.999450,0.999992,0.998851,0.983352,0.991041,0.025394
0,LG_test,0.968820,0.702908,0.125000,0.001293,0.002559,0.196785


In [ ]:
# Создаем и обучаем линейную регрессию
model = LogisticRegression(class_weight='balanced', random_state=RAND)
model.fit(x_train, y_train)

In [32]:
y_pred_tr = model.predict(x_train)
y_score_tr = model.predict_proba(x_train)

y_pred = model.predict(x_test)
y_score = model.predict_proba(x_test)

In [33]:
res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, "LG_train"))
res = res.append(get_metrics(y_test, y_pred, y_score, "LG_test"))
res

/tmp/ipykernel_7215/3198375833.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, "LG_train"))
/tmp/ipykernel_7215/3198375833.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_test, y_pred, y_score, "LG_test"))


,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,lgbm_train,0.855370,0.956167,0.166378,0.916438,0.281627,0.337983
0,lgbm_test,0.828300,0.708052,0.079067,0.427279,0.133441,0.380872
0,catboost_train,0.970535,0.825089,1.000000,0.047519,0.090727,0.110776
0,catboost_test,0.968860,0.756799,0.000000,0.000000,0.000000,0.124916
0,LG_train,0.999450,0.999992,0.998851,0.983352,0.991041,0.025394
0,LG_test,0.968820,0.702908,0.125000,0.001293,0.002559,0.196785
0,LG_train,0.692395,0.761723,0.068633,0.711492,0.125190,0.592250
0,LG_test,0.687520,0.746272,0.065980,0.691661,0.120468,0.599904


**Выводы**  
Среди выбранных baseline-моделей catboost показал большое переобучение, что можно нивилировать, подобрав гиперпараметры; lightgbm показал меньше переобучение и большое значение ROC-AUC на валидации; логистическая регрессия показала слишком низкие метрики и на трейне и на валидации, что может говорить о том, что линейная модель не подходит для данной задачи.
  
Так как в нашем случае матрица объект-признаки имеет большую размерность, lightgbm является более подходящей моделью для обучения на стационарном устройстве, так как сам алгоритм работает быстрее.

# Tune

In [3]:
import optuna
import scipy.stats as stats

In [4]:
def objective_gpt(trial: optuna.Trial, x, y):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [2000]),
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 3),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.09669864122128755]),
        'max_bin': trial.suggest_int('max_bin', 10, 500),
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 100, 70000, step=100),
        'lambda_l1': trial.suggest_int('lambda_l1', 0, 100),
        'lambda_l2': trial.suggest_int('lambda_l2', 0, 100),
        'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-4, 1e-1),
        'objective': trial.suggest_categorical('objective', ['binary']),
        'metric': trial.suggest_categorical('metric', ['auc']),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'random_state': trial.suggest_categorical('random_state', [RAND]),
    }

    cv_pred = np.empty(N_FOLDS)
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)

    for fold, (train_idx, test_idx) in enumerate(cv.split(x, y)):
        x_train_, x_val_ = x[train_idx, :], x[test_idx, :]
        y_train_, y_val_ = y[train_idx], y[test_idx]

#         ratio = y_train_[y_train_ == 0].shape[0] / \
#             y_train_[y_train_ == 1].shape[0]

        pruning = optuna.integration.LightGBMPruningCallback(trial, 'auc')

        model = LGBMClassifier(
            class_weight="balanced",
            n_jobs=-1,
            **params
        )
        model.fit(x_train_, y_train_,
                  eval_metric='auc',
                  eval_set=[(x_val_, y_val_)],
                  early_stopping_rounds=100,
                  callbacks=[pruning],
                  verbose=0)

        y_pred = model.predict(x_val_)
        y_proba = model.predict_proba(x_val_)[:, 1]

        cv_pred[fold] = roc_auc_score(y_val_, y_proba)
    return (np.mean(cv_pred))

In [ ]:
func = lambda trial: objective_gpt(trial, x_train.values, y_train.values)

study = optuna.create_study(direction="maximize")
study.optimize(func, n_trials=10, show_progress_bar=True, n_jobs=4)

In [ ]:
study.optimize(func, n_trials=10, show_progress_bar=True, n_jobs=4)

In [15]:
study.best_params

{'n_estimators': 1000,
 'learning_rate': 0.09669864122128755,
 'max_bin': 498,
 'num_leaves': 634,
 'max_depth': 15,
 'min_child_samples': 4400,
 'lambda_l1': 3,
 'lambda_l2': 51,
 'min_split_gain': 0.0911450514867725,
 'objective': 'binary',
 'metric': 'auc',
 'feature_fraction': 0.961404038163165,
 'bagging_fraction': 0.9997710561171166,
 'bagging_freq': 7,
 'random_state': 42}

In [ ]:
# Посчитаем микро- и макро- усреднение roc-auc 
cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)
roc_aucs = []
y_pred_hold = []
y_score_hold = []

for fold, (train_idx, test_idx) in enumerate(cv.split(x_train.values, y_train)):
    x_train_, x_val_ = x_train.values[train_idx, :], x_train.values[test_idx, :]
    y_train_, y_val_ = y_train.values[train_idx], y_train.values[test_idx]

#     ratio = y_train_[y_train_ == 0].shape[0] / \
#         y_train_[y_train_ == 1].shape[0]

    model = LGBMClassifier(**study.best_params, class_weight='balanced', n_jobs=-1)
    model.fit(x_train_, y_train_,
              eval_metric='auc',
              eval_set=[(x_val_, y_val_)],
              early_stopping_rounds=100,
              verbose=0)

    y_score = model.predict_proba(x_val_)

    roc_aucs.append(roc_auc_score(y_val_, y_score[:, 1]))

    y_pred_hold.append(model.predict(x_test))
    y_score_hold.append(model.predict_proba(x_test))

y_pred = stats.mode(np.column_stack(y_pred_hold), axis=1)[0]
y_score = np.column_stack([np.mean(np.column_stack(y_score_hold)[:, ::2], axis=1),
                           np.mean(np.column_stack(y_score_hold)[:, 1::2], axis=1)])

In [19]:
print(np.mean(roc_aucs))
print(roc_auc_score(y_test, y_score[:, 1]))

0.7621019672621461
0.7566207710252698


In [20]:
params = study.best_params

joblib.dump(params, "../data/params.joblib")

['../data/params.joblib']

# Out-of-core training

In [4]:
from scrypt import out_of_core_train

In [5]:
train_path = "../data/transformed/train_data/"
target_path = "../data/raw/train_target.csv"

In [6]:
params = joblib.load("../data/params.joblib")

In [7]:
model = out_of_core_train(train_path, target_path, **params)

  0%|          | 0/11 [00:00<?, ?it/s]

In [8]:
# Создаем словарь с моделью
models = {}

models['model'] = model

In [9]:
# Сохраняем словарь с моделями в joblib файл
models_path = '../data/models.joblib'
joblib.dump(models, models_path)

['../data/models.joblib']